In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [ ]:
cd 'drive'/'MyDrive'/'Dacon'/'LDI'

/content/drive/MyDrive/Dacon/LDI


In [ ]:
train_err_data = pd.read_csv('./dataset/train_err_data.csv',thousands = ',')
train_quality_data = pd.read_csv('./dataset/train_quality_data.csv',thousands = ',')
train_problem_data = pd.read_csv('./dataset/train_problem_data.csv',thousands = ',')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_tqd = train_quality_data.drop_duplicates()
df_ted = train_err_data.drop_duplicates()
df_tpd = train_problem_data.drop_duplicates()

In [ ]:
from sklearn.preprocessing import LabelEncoder
encode_index = [1,5,8,9,23,25,38,41] #
df_ted_encoded = pd.DataFrame(columns=["user_id","time","model_nm","fwver","errtype","errcode"])
for i in encode_index:   # Label encoding of errtype
    temp = df_ted[df_ted.errtype == i]
    le = LabelEncoder().fit_transform(temp["errcode"].astype(str))
    temp["errcode"] = le
    df_ted_encoded = pd.concat([df_ted_encoded,temp])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [ ]:
df_ted_encoded["model_nm"] = LabelEncoder().fit_transform(df_ted_encoded["model_nm"].astype(str)) # Encoding by model number

In [ ]:
encodedfwvlst = ['04.16.3553','04.22.1750','04.33.1261','05.15.2138','04.22.1778','04.82.1684',  # Encoding ted by fwversion
                 '04.33.1185','03.11.1167','04.16.3571','04.73.2237','05.66.3237']
lst_fwv = list(df_ted_encoded['fwver'])
for i in range(len(lst_fwv)):
    if lst_fwv[i] not in encodedfwvlst:
        lst_fwv[i] = 'nope'
df_ted_encoded['fwver'] = lst_fwv
df_ted_encoded["fwver"] = LabelEncoder().fit_transform(df_ted_encoded["fwver"].astype(str))
df_ted_encoded

,user_id,time,model_nm,fwver,errtype,errcode
83389,10080,20201101030624,4,11,1,0
83481,10080,20201102035827,4,11,1,0
83482,10080,20201103052730,4,11,1,0
83493,10080,20201104023034,4,11,1,0
83494,10080,20201105050236,4,11,1,0
...,...,...,...,...,...,...
16553627,24997,20201120211047,0,4,41,0
16553629,24997,20201120211048,0,4,41,0
16553630,24997,20201120211049,0,4,41,0
16553631,24997,20201120225907,0,4,41,0


In [ ]:
encodedfwvlst = ['04.16.3553','04.22.1750','04.33.1261','05.15.2138','04.22.1778','04.82.1684', # Encoding tqd by fwversion
                 '04.33.1185','03.11.1167','04.16.3571','04.73.2237','05.66.3237']
lst_fwv = list(df_tqd['fwver'])
for i in range(len(lst_fwv)):
    if lst_fwv[i] not in encodedfwvlst:
        lst_fwv[i] = 'nope'
df_tqd['fwver'] = lst_fwv
print(df_tqd['fwver'].unique())
df_tqd["fwver"] = LabelEncoder().fit_transform(df_tqd["fwver"].astype(str))

['05.15.2138' '04.22.1750' '04.16.3553' '04.33.1261' '04.22.1778'
 '04.33.1185' '04.16.3571' 'nope' '03.11.1167' '04.82.1684' '05.66.3237'
 '04.73.2237']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
lst_column = ["user_id", "model_nm", "fwver"]  # Make a output datafarme

for i in range(1,43):
    if i == 29:
        continue
    lst_column.append('errtype_' + str(i))
for i in range(0,13):
    lst_column.append('quality_' + str(i))

df_x = pd.DataFrame(columns = lst_column)
df_x

,user_id,model_nm,fwver,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


In [ ]:
for i in range(len(df_tpd)):
    print(str(i),"/",str(len(df_tpd)),"finished.")
    time = df_tpd.loc[i].time
    user_id = df_tpd.loc[i].user_id
    df_check_time_ted = df_ted_encoded[(df_ted_encoded.user_id == user_id) & (df_ted_encoded.time <= time)]
    df_check_time_tqd = df_tqd[(df_tqd.user_id == user_id) & (df_tqd.time <= time)]
    fwvlst = list(df_check_time_ted['fwver'].unique())
    for fwv in fwvlst:
        df_check_fwv_ted = df_check_time_ted[df_check_time_ted.fwver == fwv]
        df_check_fwv_tqd = df_check_time_tqd[df_check_time_tqd.fwver == fwv]
        df_check_fwv_tqd = df_check_fwv_tqd.drop('time', axis = 1)
        df_mean_quality_tqd = df_check_fwv_tqd.mean()
        df_mean_quality_tqd = df_mean_quality_tqd.fillna(0)
        mnlst = list(df_check_fwv_ted['model_nm'].unique())
        for mn in mnlst:
            df_check_mn_ted = df_check_fwv_ted[df_check_fwv_ted.model_nm == mn]
            df_check_mn_ted = df_check_mn_ted.drop('time', axis = 1)
            errtypelst = list(df_check_mn_ted['errtype'].unique())
            errtype_val = {}
            for errtype in errtypelst:
                df_check_ert_ted = df_check_mn_ted[df_check_mn_ted.errtype == errtype]
                df_mean_erc_ted = df_check_ert_ted['errcode'].mean()
                errtype_val[errtype] = df_mean_erc_ted
            result_row = [user_id,mn,fwv]
            e_v_keys = errtype_val.keys()
            for j in range(1,43):
                if j == 29:
                    continue
                if j in e_v_keys:
                    result_row.append(errtype_val[j])
                else:
                    result_row.append(0)
            for j in range(2,15):
                result_row.append(df_mean_quality_tqd[j])
            df_x = df_x.append(pd.Series(result_row,index = df_x.columns),ignore_index=True)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
429 / 5429 finished.
430 / 5429 finished.
431 / 5429 finished.
432 / 5429 finished.
433 / 5429 finished.
434 / 5429 finished.
435 / 5429 finished.
436 / 5429 finished.
437 / 5429 finished.
438 / 5429 finished.
439 / 5429 finished.
440 / 5429 finished.
441 / 5429 finished.
442 / 5429 finished.
443 / 5429 finished.
444 / 5429 finished.
445 / 5429 finished.
446 / 5429 finished.
447 / 5429 finished.
448 / 5429 finished.
449 / 5429 finished.
450 / 5429 finished.
451 / 5429 finished.
452 / 5429 finished.
453 / 5429 finished.
454 / 5429 finished.
455 / 5429 finished.
456 / 5429 finished.
457 / 5429 finished.
458 / 5429 finished.
459 / 5429 finished.
460 / 5429 finished.
461 / 5429 finished.
462 / 5429 finished.
463 / 5429 finished.
464 / 5429 finished.
465 / 5429 finished.
466 / 5429 finished.
467 / 5429 finished.
468 / 5429 finished.
469 / 5429 finished.
470 / 5429 finished.
471 / 5429 finished.
472 / 5429 finished.
473 / 5429 finished.
474 / 5429 finished

In [ ]:
df_x

,user_id,model_nm,fwver,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,19224.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
1,23664.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.111111,-0.111111,-0.111111,0.0,0.0,0.888889,-0.111111,0.0,0.0,0.0,3.222222,-0.111111,0.0
2,23664.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.285714,-0.285714,-0.285714,0.0,0.0,0.428571,-0.285714,0.0,0.0,0.0,2.571429,-0.285714,0.0
3,15166.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.910508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.250000,-0.250000,-0.250000,0.0,0.0,0.500000,-0.250000,0.0,0.0,0.0,3.000000,-0.250000,0.0
4,12590.0,1.0,1.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6234,20167.0,0.0,3.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
6235,20167.0,0.0,4.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
6236,16270.0,2.0,5.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
6237,19114.0,3.0,9.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0


In [ ]:
import pickle
with open('list.txt', 'wb') as f:
    pickle.dump(df_x, f)